In [1]:
import pandas as pd
import s3fs
import boto3
import json
from datetime import datetime

In [2]:
fs = s3fs.S3FileSystem()
s3 = boto3.client('s3')
fnames = fs.ls('s3://streetview-w210/gsv/')
fnames = [x for x in fnames if x.endswith('.json')]
len(fnames)

19325

In [3]:
bucket_name = 'streetview-w210'
dates, lat, long, name, pano_id = [],[],[],[],[]

start = datetime.now()

for i,fname in enumerate(fnames):
    json_path = 's3://{}'.format(fname)
    obj = s3.get_object(Bucket=bucket_name, Key=fname[len(bucket_name)+1:])
    json_file = obj['Body'].read().decode('utf-8')
    
    file_data = json.loads(json_file)
    if 'date' not in file_data:
        print(fname)
        continue
    dates.append(file_data['date'])
    lat.append(file_data['location']['lat'])
    long.append(file_data['location']['lng'])
    pano_id.append(file_data['pano_id'])
    name.append(fname[len(bucket_name)+1:])
    
    
print(datetime.now()-start)

df = pd.DataFrame([dates,lat,long,pano_id,name]).transpose()
df.columns = ['date','lat','long','pano_id','name']

0:10:52.121497


In [5]:
df['name'] = [x[4:] for x in df['name']]

In [6]:
df.to_csv('gsv_metadata.csv')